# Как работать с датами в pandas
https://habr.com/ru/companies/wunderfund/articles/677068/

In [43]:
import pandas as pd
import numpy as np
df = pd.DataFrame({
    "booking_id": [1001, 1002, 1003, 1004, 1005],
    "property" : ["A", "A", "B", "B", "C"],
    "created_at": ["2022-03-01", "2022-02-10", "2022-04-12",
                   "2022-04-11", "2022-06-05"],
    "checkin_date": ["2022-06-01", "2022-06-10", "2022-06-02",
                     "2022-06-20", "2022-08-10"],
    "checkout_date": ["2022-06-06", "2022-06-15", 
                      "2022-06-06","2022-06-28", "2022-08-16"],
    "amount": [5400, 5600, 4800, 9000, 6500]
})
df

,booking_id,property,created_at,checkin_date,checkout_date,amount
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500


In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   booking_id     5 non-null      int64 
 1   property       5 non-null      object
 2   created_at     5 non-null      object
 3   checkin_date   5 non-null      object
 4   checkout_date  5 non-null      object
 5   amount         5 non-null      int64 
dtypes: int64(2), object(4)
memory usage: 372.0+ bytes


## Изменение типа данных на 'datetime64[ns]'

In [48]:
date_cols = ['created_at', 'checkin_date', 'checkout_date']
df[date_cols] = df[date_cols].astype('datetime64[ns]')
df

,booking_id,property,created_at,checkin_date,checkout_date,amount
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500


In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   booking_id     5 non-null      int64         
 1   property       5 non-null      object        
 2   created_at     5 non-null      datetime64[ns]
 3   checkin_date   5 non-null      datetime64[ns]
 4   checkout_date  5 non-null      datetime64[ns]
 5   amount         5 non-null      int64         
dtypes: datetime64[ns](3), int64(2), object(1)
memory usage: 372.0+ bytes


## 1. Как извлечь из даты информацию о месяце и годе?

Используем методы, к которым обращаемся через аксессор `dt`

### `dt.month` - получить номер месяца

In [54]:
df['checkin_date'].dt.month

0    6
1    6
2    6
3    6
4    8
Name: checkin_date, dtype: int32

### `.dt.day` - получить дату

In [57]:
df['checkin_date'].dt.day

0     1
1    10
2     2
3    20
4    10
Name: checkin_date, dtype: int32

### `.dt.year` - получить год

In [60]:
df['checkin_date'].dt.year

0    2022
1    2022
2    2022
3    2022
4    2022
Name: checkin_date, dtype: int32

### `dt.to_period('M')` - получить `ГГГГ-ММ`

In [63]:
df['year_month'] = df['checkin_date'].dt.to_period('M')
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-06
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-06
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-06
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-06
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-08


## 2. Как прибавить к дате временной интервал?
Используем функцию `DateOffset`

Добавим 1 день к дате выселения (например — из некоего объекта недвижимости) (checkout_date) для записи о бронировании с идентификатором 1001:

In [73]:
df.loc[df['booking_id']==1001, 'checkout_date_upd'] = df.loc[df['booking_id']==1001, 'checkout_date'] + \
                                                    pd.DateOffset(days=1)

df              

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,checkout_date_upd
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-06,2022-06-07
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-06,NaT
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-06,NaT
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-06,NaT
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-08,NaT


## 3. Как найти интервал в днях между двумя датами?
Интервал между двумя датами можно найти, вычтя одну из другой. 
Результатом этой операции будет объект `Timedelta`, а это не совсем то, что нам нужно:

In [78]:
df['checkout_date_upd'][0] - df['checkin_date'][0]

Timedelta('6 days 00:00:00')

Получить разницу между датами в днях, в виде целого числа, можно, воспользовавшись атрибутом `days`.

In [81]:
df['days_to_checkin'] = (df['checkin_date'] - df['created_at'])
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,checkout_date_upd,days_to_checkin
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-06,2022-06-07,92 days
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-06,NaT,120 days
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-06,NaT,51 days
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-06,NaT,70 days
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-08,NaT,66 days


In [83]:
df['days_to_checkin'] = (df['checkin_date'] - df['created_at']).dt.days
df

,booking_id,property,created_at,checkin_date,checkout_date,amount,year_month,checkout_date_upd,days_to_checkin
0,1001,A,2022-03-01,2022-06-01,2022-06-06,5400,2022-06,2022-06-07,92
1,1002,A,2022-02-10,2022-06-10,2022-06-15,5600,2022-06,NaT,120
2,1003,B,2022-04-12,2022-06-02,2022-06-06,4800,2022-06,NaT,51
3,1004,B,2022-04-11,2022-06-20,2022-06-28,9000,2022-06,NaT,70
4,1005,C,2022-06-05,2022-08-10,2022-08-16,6500,2022-08,NaT,66


4. Как сформировать последовательность дат между начальной и конечной датами?

+ Предположим, нам нужен календарь, который показывает дни, в которые объект недвижимости забронирован.
+ Сейчас в первой строке датафрейма есть запись о том, что объект недвижимости A забронирован с 2022–06–01 по 2022–06–07.
+ Получается, что объект A забронирован на даты 2022–06–01, 2022–06–02, 2022–06–03, 2022–06–04, 2022–06–05, 2022–06–06 (если предположить, что постоялец выселяется в 10 утра 2022–06–07).

Такой календарь можно создать, найдя даты между датами вселения и выселения и внеся их в объект DataFrame.

Для начала создадим датафрейм calendar, содержащий столбцы property, checkin_date и checkout_date:

In [88]:
calendar = df[['property', 'checkin_date', 'checkout_date']]
calendar

,property,checkin_date,checkout_date
0,A,2022-06-01,2022-06-06
1,A,2022-06-10,2022-06-15
2,B,2022-06-02,2022-06-06
3,B,2022-06-20,2022-06-28
4,C,2022-08-10,2022-08-16


Функция `date_range` даёт нам даты, находящиеся между начальной и конечной датами

In [91]:
pd.date_range(calendar['checkin_date'][0], calendar['checkout_date'][0])

DatetimeIndex(['2022-06-01', '2022-06-02', '2022-06-03', '2022-06-04',
               '2022-06-05', '2022-06-06'],
              dtype='datetime64[ns]', freq='D')

Тут есть одна проблема: 
+ нам не нужно, чтобы дата выселения (`checkout_date`) была бы показана как дата, когда объект недвижимости забронирован.
+ Поэтому мы, прежде чем находить диапазон дат, вычтем 1 из даты выселения.

Для того чтобы сделать это для всех строк, нужно воспользоваться функцией `apply`. 
+ Мы, кроме того, конвертируем вывод функции date_range в список, воспользовавшись конструктором `list`:

In [96]:
calendar.loc[:, 'booked_days'] = calendar.apply(

    lambda x: list(
        pd.date_range(
            x['checkin_date'],
            x['checkout_date'] - pd.DateOffset(days=1)
        ).date
    ),
    axis=1
)
calendar

C:\Users\A.G. Shurakov\AppData\Local\Temp\ipykernel_10888\4041795568.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  calendar.loc[:, 'booked_days'] = calendar.apply(


,property,checkin_date,checkout_date,booked_days
0,A,2022-06-01,2022-06-06,"[2022-06-01, 2022-06-02, 2022-06-03, 2022-06-0..."
1,A,2022-06-10,2022-06-15,"[2022-06-10, 2022-06-11, 2022-06-12, 2022-06-1..."
2,B,2022-06-02,2022-06-06,"[2022-06-02, 2022-06-03, 2022-06-04, 2022-06-05]"
3,B,2022-06-20,2022-06-28,"[2022-06-20, 2022-06-21, 2022-06-22, 2022-06-2..."
4,C,2022-08-10,2022-08-16,"[2022-08-10, 2022-08-11, 2022-08-12, 2022-08-1..."


Далее — нам нужно заполнить датафрейм, развернув даты из столбца `booked_days`. 

В этом нам поможет функция `explode`:

In [99]:
calendar = calendar.explode(column='booked_days', ignore_index=True
                           )[['property', 'booked_days']]
calendar.head(30)

,property,booked_days
0,A,2022-06-01
1,A,2022-06-02
2,A,2022-06-03
3,A,2022-06-04
4,A,2022-06-05
5,A,2022-06-10
6,A,2022-06-11
7,A,2022-06-12
8,A,2022-06-13
9,A,2022-06-14
